In [ ]:
from dataclasses import dataclass
import json

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from skimage.measure import block_reduce
from skimage.transform import resize

In [ ]:
def discretize_extent(tile_extent: tuple[tuple[float, float], tuple[float, float]],
                      tile_shape: tuple[int, int],
                      requested_extent: tuple[tuple[float, float], tuple[float, float]]):
    # axis 0 is latitude north->south (!)
    # axis 1 is longitude west->east

    (tile_min_lat, tile_min_lon), (tile_max_lat, tile_max_lon) = tile_extent
    relative_extent = ((requested_extent[0][0] - tile_min_lat, requested_extent[0][1] - tile_min_lon),
                       (requested_extent[1][0] - tile_min_lat, requested_extent[1][1] - tile_min_lon))

    print(f"{relative_extent=}")

    # project user extents into pixel space
    px_per_deg_lat = tile_shape[0] / (tile_max_lat - tile_min_lat)
    px_per_deg_lon = tile_shape[1] / (tile_max_lon - tile_min_lon)

    print(f"{px_per_deg_lat=} {px_per_deg_lon=}")

    min_y = int(np.floor(relative_extent[0][0] * px_per_deg_lat))
    min_x = int(np.floor(relative_extent[0][1] * px_per_deg_lon))
    max_y = int(np.ceil(relative_extent[1][0] * px_per_deg_lat))
    max_x = int(np.ceil(relative_extent[1][1] * px_per_deg_lon))

    print(f"{min_y=} {min_x=} {max_y=} {max_x=}")
    assert min_y >= 0 and min_y <= tile_shape[0]
    assert min_x >= 0 and min_x <= tile_shape[1]
    assert max_y >= 0 and max_y <= tile_shape[0]
    assert max_x >= 0 and max_x <= tile_shape[1]

    # reconstruct geo coordinates
    discretized_extent = ((tile_min_lat + min_y / px_per_deg_lat, tile_min_lon + min_x / px_per_deg_lon),
                          (tile_min_lat + max_y / px_per_deg_lat, tile_min_lon + max_x / px_per_deg_lon))
    print(f"{discretized_extent=}")

    # flip Y
    min_y_corr = tile_shape[0] - max_y
    max_y_corr = tile_shape[0] - min_y
    print(f"{min_y_corr=} {max_y_corr=}")
    assert min_y_corr >= 0 and min_y_corr <= tile_shape[0]
    assert max_y_corr >= 0 and max_y_corr <= tile_shape[0]

    return ((min_y_corr, max_y_corr), (min_x, max_x)), discretized_extent

In [ ]:
def load_unzipped(coords):
    # dataset is provided in the form of tiles 3x3 degrees in size
    # compute tile coordinates
    tile_coords = (int((coords[0] // 3) * 3), int((coords[1] // 3) * 3))

    tile_extent = (tile_coords, (tile_coords[0] + 3, tile_coords[1] + 3))

    # Build macrotile archive name from macrotile coords.
    # Archive naming uses hemisphere prefixes: N30E120 etc. Determine strings:
    lat_prefix = f"N{abs(tile_coords[0]):02d}" if tile_coords[0] >= 0 else f"S{abs(tile_coords[0]):02d}"
    lon_prefix = f"E{int(tile_coords[1]):03d}" if tile_coords[1] < 180 else f"W{int((360-tile_coords[1])%360):03d}"

    filename = f"inputs/ESA_WorldCover_10m_2021_V200_{lat_prefix}{lon_prefix}_Map.tif"
    print("load", filename)

    with rasterio.open(filename) as dataset:
        data = dataset.read(1)  # read first band
        profile = dataset.profile
    return data, profile, tile_extent

# 4.2s to load TIF
# 2.6s to load gzipped pickle --> not worth the savings
tile_data, profile, tile_extent = load_unzipped((35.48556194027054, 139.86056640390504))

print(f"{tile_data.nbytes=}")
print(f"{profile=}")
print(f"{tile_extent=}")

In [ ]:
# extract region
coords, discretized_extent = discretize_extent(tile_extent=tile_extent,
                                               tile_shape=tile_data.shape,
                                               requested_extent=((34.75, 139.0), (36, 141)))



#return ((min_y_corr, max_y_corr + 1), (min_x, max_x + 1)), discretized_extent

# extract just a small area of the 36000x36000 image for display
# axis 0 is north->south
# axis 1 is west->east
D = 30  # initial decimation factor (another round of decimation is done later)
extract_data = tile_data[coords[0][0]:coords[0][1]:D, coords[1][0]:coords[1][1]:D][::-1]
#extract_data = tile_data[:15000:D, 18000::D].copy()
print(f"{extract_data.shape=}")

f, ax = plt.subplots(figsize=(10, 10))
cmap = ListedColormap([
    "#000000",  # No data
    "#006400",  # Tree cover
    "#ffbb22",  # Shrubland
    "#ffff4c",  # Grassland
    "#f096ff",  # Cropland
    "#fa0000",  # Built-up
    "#b4b4b4",  # Bare / sparse vegetation
    "#f0f0f0",  # Snow and ice
    "#0064c8",  # Permanent water bodies
    "#0096a0",  # Herbaceous wetland
    "#00cf75",  # Mangroves
    "#fae6a0",  # Moss and lichen
])
im = ax.imshow(extract_data // 10, origin="lower", cmap=cmap, vmin=0, vmax=11)
cbar = f.colorbar(im, ax=ax, fraction=0.036, pad=0.04, ticks=np.arange(0, 12))
cbar.ax.set_yticklabels([
    "No data",
    "Tree cover",
    "Shrubland",
    "Grassland",
    "Cropland",
    "Built-up",
    "Bare / sparse vegetation",
    "Snow and ice",
    "Permanent water bodies",
    "Herbaceous wetland",
    "Mangroves",
    "Moss and lichen",
])
None

In [ ]:
NO_DATA = 0
WATER = 80
water_mask_large = np.logical_or(extract_data == NO_DATA, extract_data == WATER).astype(np.uint8)

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask_large, origin="lower", cmap="Blues")

In [ ]:
# def decimate(extract_data, factor):
#     assert len(extract_data.shape) == 2
#     assert extract_data.shape[0] % factor == 0
#     assert extract_data.shape[1] % factor == 0

#     fac0, fac1 = factor, factor
#     is0, is1 = extract_data.shape
#     os0, os1 = is0 // fac0, is1 // fac1

#     output = np.zeros(shape=(os0, os1), dtype=np.float32)

#     for i, j in np.ndindex((os0, os1)):
#         block = extract_data[i*fac0:(i+1)*fac0, j*fac1:(j+1)*fac1]
#         output[i, j] = np.mean(block)

#     return output

# use this trick: https://stackoverflow.com/a/26639111
DECIMATION_FACTOR = 10
assert water_mask_large.shape[0] % DECIMATION_FACTOR == 0
assert water_mask_large.shape[1] % DECIMATION_FACTOR == 0
water_mask_decimated = block_reduce(water_mask_large, block_size=DECIMATION_FACTOR, func=np.mean)

# water_mask = decimate(water_mask, DECIMATION_FACTOR)
print(f"{water_mask_decimated.shape=}")

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask_decimated, origin="lower", cmap="Blues")

#
THRESHOLD = 0.50
water_mask = (water_mask_decimated >= THRESHOLD).astype(np.uint8)

f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask, origin="lower", cmap="Blues")

In [ ]:
try:
    height_data = np.load("height_data.npy")
except FileNotFoundError:
    import sys
    sys.path.insert(0, "/home/mce/dev/CLOU25/")
    import elevmodel

    import logging

    logging.basicConfig()
    logging.getLogger("elevmodel").setLevel(logging.DEBUG)

    # model = elevmodel.NASADEM_HGT("/mnt/nas/public/Datasets/NASADEM_HGT.001", format="zip")
    model = elevmodel.SRTMGL1("/mnt/nas/public/Datasets/SRTMGL1", format="zip")

    height_data, _, _, _, _ = elevmodel.extract(model, *discretized_extent[0], *discretized_extent[1])

    np.save("height_data.npy", height_data)

In [ ]:
# 20sec to compute
height_data_resized = resize(height_data, water_mask.shape, preserve_range=True, order=1, mode="edge", anti_aliasing=True)

height = height_data_resized.copy()
height[water_mask > 0] = -100

# plot height_data_resized
f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(height, origin="lower")
cbar = f.colorbar(im, ax=ax, fraction=0.036, pad=0.04)
cbar.ax.set_ylabel("Height (m)")

In [ ]:
with open("japan_cities_in_extent.json") as f:
    cities = json.load(f)

# plot data
f, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(water_mask, origin="lower", cmap="Blues")

cities = list(sorted(cities, key=lambda c: c["population"], reverse=True))

# TODO: go largest -> smallest, accept each only if clear space around

@dataclass
class Town:
    id: int
    name: str
    x: int
    y: int
    pop: int

towns = []

MIN_TOWN_DIST = 3
TOWN_DIST_FACTOR = 0.03
TOWN_DISPLAY_SCALE = 10
MOUNTAIN_THR = 120

for c in cities:
    pop = c["population"] // 1000
    x = (c["longitude"] - discretized_extent[0][1]) / (discretized_extent[1][1] - discretized_extent[0][1]) * water_mask.shape[1]
    y = (c["latitude"] - discretized_extent[0][0]) / (discretized_extent[1][0] - discretized_extent[0][0]) * water_mask.shape[0]
    x = int(np.round(x))
    y = int(np.round(y))
    # ax.text(s=f'{c["asciiname"]} ({pop}k)', x=x, y=y)

    if y >= height.shape[0] or height[y, x] < 0 or height[y, x] > MOUNTAIN_THR:
        continue

    too_close = False

    for t in towns:
        min_dist = MIN_TOWN_DIST + (np.sqrt(t.pop) + np.sqrt(pop)) * TOWN_DIST_FACTOR
        if (t.x - x) ** 2 + (t.y - y) ** 2 < min_dist ** 2:
            print(f"Reject {c['asciiname']} ({pop}k) too close to {t.name} ({t.pop}k) min_dist={min_dist:.1f} actual_dist={( (t.x - x) ** 2 + (t.y - y) ** 2 )**0.5:.1f}")
            too_close = True
            break

    if too_close:
        continue

    towns.append(Town(id=len(towns), name=c["asciiname"], x=x, y=y, pop=pop))
    ax.text(s=f'{c["asciiname"]} ({pop}k)', x=x, y=y)

    area = np.sqrt(pop) * TOWN_DISPLAY_SCALE       # MPL uses points squared as the unit of size for scatter
    ax.scatter(x, y, s=area, c="red", alpha=0.5, edgecolors="black", linewidths=0.5)

print(f"{len(towns)=}")
plt.show()

In [ ]:
# with open("japan.json", "w") as f:
#     lake_map = (water_mask > 0).astype(np.uint8)
#     json.dump({
#         "height": height.tolist(),
#         "towns": [{"id": t.id, "name": t.name, "x": t.x, "y": t.y, "pop": t.pop} for t in towns]
#     }, f)

In [ ]:
# Create tile map in Tiled JSON format
tilemap = {
    "compressionlevel": -1,
    "height": water_mask.shape[0],
    "width": water_mask.shape[1],
    "infinite": False,
    "layers": [
        {
            # Terrain layer
            "data": [3 if height[y, x] > MOUNTAIN_THR else (2 if water_mask[y,x] > 0 else 1)
                    for y in range(water_mask.shape[0]-1, -1, -1)
                    for x in range(water_mask.shape[1])],
            "height": water_mask.shape[0],
            "width": water_mask.shape[1],
            "id": 1,
            "name": "Terrain",
            "opacity": 1,
            "type": "tilelayer",
            "visible": True,
            "x": 0,
            "y": 0
        },
        {
            # Towns layer
            "draworder": "topdown",
            "id": 2,
            "name": "Towns",
            "objects": [
                {
                    "gid": 4,  # Town tile ID
                    "height": 16,
                    "id": t.id + 1,
                    "name": t.name,
                    "properties":[
                            {
                            "name":"population",
                            "type":"int",
                            "value":t.pop,
                            }],
                    "rotation": 0,
                    "visible": True,
                    "width": 16,
                    "x": t.x * 16,
                    "y": (water_mask.shape[0] - t.y) * 16  # Flip Y coordinate
                }
                for t in towns
            ],
            "opacity": 1,
            "type": "objectgroup",
            "visible": True,
            "x": 0,
            "y": 0
        }
    ],
    "nextlayerid": 3,
    "nextobjectid": len(towns) + 1,
    "orientation": "orthogonal",
    "renderorder": "right-up",
    "tiledversion": "1.11.2",
    "tileheight": 16,
    "tilewidth": 16,
    "tilesets": [
        {
            "columns": 4,
            "firstgid": 1,
            "image": "tileset.png",
            "imageheight": 16,
            "imagewidth": 64,
            "margin": 0,
            "name": "tileset",
            "spacing": 0,
            "tilecount": 4,
            "tileheight": 16,
            "tilewidth": 16
        }
    ],
    "type": "map",
    "version": "1.10"
}

# Save to file
with open("japan.tmj", "w") as f:
    json.dump(tilemap, f)